<a href="https://colab.research.google.com/github/jiangxueqi/LLM-quickstart/blob/main/homework/homework2_1_%E4%BD%BF%E7%94%A8%E5%AE%8C%E6%95%B4%E6%95%B0%E6%8D%AE%E9%9B%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -r "/content/drive/MyDrive/Colab Notebooks/AI大模型微调训练营/6_实战Transformers模型微调/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 

#1、下载数据集

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

# 随机抽取样本展示
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

,label,text
0,5 stars,"Our dinner at Sage was absolutely heavenly. Hands down one of my favorite meals ever. The d\u00e9cor is chic. and parts of the walls were lined with blown-up images of what reminded me of French Impressionist paintings. For starters, I highly recommend the fois gras cr\u00e8me brulee, and sweetbread. I'm not a big fois gras connoisseur, but I loved it in its cr\u00e8me brulee reincarnation. It is served with toasted brioche, which our waiter brought a second round since we devoured our first batch so quickly. The sweetbread dish had probably the best melt-in-my-mouth pork belly I've ever had. Perfect if you love strong savory flavors. For my main course I had the braised beef, and it was flawless. Tender tender.\n\nI did not want this meal to end.\n\nNaturally, I tried to drag this meal out as long as I could, so I got dessert. I had the deconstructed carrot \""cake.\"" Very tasty but having a sweet tooth like mine, I thought it was a bit small. My boyfriend ended his meal with absinthe prepared the traditional way, complete with flaming sugar cube and all. Our waitress was very attentive, knowledgeable, and spot on with her recommendations. Great service overall.\n\nSage is not to be missed when in Vegas."
1,3 stars,"Decent diner food with tasty pies. It's kind of like a more upscale version of Denny's, but with more selection. I like to come here on Pie Rush Wednesdays, where you can get a free slice of pie with dinner purchase. They currently have a seasonal Hawaiian strawberry pie that is pretty tasty! As for their breakfasts, generally I've been pretty satisfied with their skillet meals."
2,5 stars,"Without a doubt the coolest bar in vegas, the most awesome bartenders, the best machines ive played in vegas, and you can get the best pizza, wings, and sandwiches in vegas all in the same place. LOVE them"
3,4 stars,"The food is decent, not as good as before the move, but still good. They get bonus points for being open late, which is always a good thing."
4,4 stars,A solid pizza joint best for take out. Every bite resembles utopia in a slice of pizza. Lovingly made by hand with the freshest ingredients and not your chain pizza place. \nOnly downside to report is not enough variety and not really a great dine in area but otherwise excellent. The reason they don't drown the pizza in their homemade sauce is because they want you to taste the ingredients and we find that refreshing and love that about them. It's definitely a must try place and who knew Mexican pizza was so delicious!
5,2 star,"This review is for the Dollar Rent a Car location at the MGM Grand:\n\nWe did a day-long rental from Dollar Rent a Car so that we could drive ourselves out to the Hoover Dam from the MGM Grand in Las Vegas. It was convenient to have the agency right in the main lobby of the hotel; that's the main reason we decided to rent from here.\n\nFirst the good: Our blue Ford Fusion was clean and well maintained. It was pretty easy to get to the cars, up on floor 5 of the MGM parking garage. \n\nNow, the bad: Both on key pickup and key return, I could not believe that there was only one attendant working the rental desk in the MGM lobby. It took almost an hour of waiting in line for us to get our keys. I had to wait for at least 20 minutes upon key return before the employee asked if anyone in line was waiting to simply drop their keys off.\n\nThe experience would have been a lot better if they thought about their staffing needs, especially during the busy weekend times. I don't know if there are better options elsewhere, but in the future I'd definitely consider just renting from the airport, even if it meant I had to take a cab over there to get the car."
6,1 star,"One of the worst hotels that wasn't a 1-2 star I've been to in a bit. \n\nGood:the bars, sports books, and casino are all top of the line. \n\nBad: I had to check my bag until 4 pm because they didn't have any rooms. I was ok with that. When I came back to get my b

#2、预处理数据

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)  # 增加了3个维度（input_ids、token_type_ids、attention_mask）

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,2 star,"Went there for the first time last night. Took 10 minutes to get my beer, but the wife got her soda right away. Waiter brought the wrong appetizer. Advised him of this after about 5 minutes (because he never came back). He realized he gave us another table's appetizer. He took the appetizer off our table and gave it to the right table (I could not believe he did that). Then our food came out before our appetizer. It was very loud and service was not good. Food was average. Not a big menu, a little pricy, and small space. Was the first and last time we go there. Nothing special.","[101, 23158, 1204, 1175, 1111, 1103, 1148, 1159, 1314, 1480, 119, 6466, 1377, 1275, 1904, 1106, 1243, 1139, 5298, 117, 1133, 1103, 1676, 1400, 1123, 18402, 1268, 1283, 119, 7911, 1200, 1814, 1103, 2488, 12647, 26883, 6198, 119, 24930, 16641, 1181, 1140, 1104, 1142, 1170, 1164, 126, 1904, 113, 1272, 1119, 1309, 1338, 1171, 114, 119, 1124, 2788, 1119, 1522, 1366, 1330, 1952, 112, 188, 12647, 26883, 6198, 119, 1124, 1261, 1103, 12647, 26883, 6198, 1228, 1412, 1952, 1105, 1522, 1122, 1106, 1103, 1268, 1952, 113, 146, 1180, 1136, 2059, 1119, 1225, 1115, 114, 119, 1599, 1412, 2094, 1338, 1149, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


In [ ]:
# 抽样数据集
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
# 全体数据集
all_train_dataset = tokenized_datasets["train"]
all_eval_dataset = tokenized_datasets["test"]

#3、配置模型、参数、评估指标

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)  # 5个中取最大的logits作为输出
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

batch_size=16

small_model_dir = "/content/drive/MyDrive/model/LLM_quickstart/6_Transformers微调实战/bert-base-cased/small"
all_model_dir = "/content/drive/MyDrive/model/LLM_quickstart/6_Transformers微调实战/bert-base-cased/all"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
small_training_args = TrainingArguments(output_dir=f"{small_model_dir}/test_trainer",
                                        evaluation_strategy="epoch",
                                        logging_dir=f"{small_model_dir}/test_trainer/runs",
                                        per_device_train_batch_size=batch_size,
                                        per_device_eval_batch_size=batch_size,
                                        num_train_epochs=3,
                                        logging_steps=100)

all_training_args = TrainingArguments(output_dir=f"{all_model_dir}/test_trainer",
                                      evaluation_strategy="epoch",
                                      logging_dir=f"{all_model_dir}/test_trainer/runs",
                                      per_device_train_batch_size=batch_size,
                                      per_device_eval_batch_size=batch_size,
                                      num_train_epochs=3,
                                      logging_steps=100)

#4、开始训练

In [ ]:
small_trainer = Trainer(
    model=model,
    args=small_training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

all_trainer = Trainer(
    model=model,
    args=all_training_args,
    train_dataset=all_train_dataset,
    eval_dataset=all_eval_dataset,
    compute_metrics=compute_metrics,
)

#4.1、抽样数据集上的Acc

In [ ]:
small_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.089913,0.517000
2,1.269600,0.990862,0.583000
3,1.269600,0.978394,0.603000


TrainOutput(global_step=189, training_loss=1.0082926068987166, metrics={'train_runtime': 120.3745, 'train_samples_per_second': 24.922, 'train_steps_per_second': 1.57, 'total_flos': 789354427392000.0, 'train_loss': 1.0082926068987166, 'epoch': 3.0})

In [ ]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))

In [ ]:
small_trainer.evaluate(small_test_dataset)

{'eval_loss': 1.0628308057785034,
 'eval_accuracy': 0.52,
 'eval_runtime': 1.0483,
 'eval_samples_per_second': 95.392,
 'eval_steps_per_second': 6.677,
 'epoch': 3.0}

#4.2、完整数据集上的Acc

In [ ]:
all_trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
all_test_dataset = tokenized_datasets["test"]

In [ ]:
all_trainer.evaluate(all_test_dataset)

In [ ]:
!apt install psmisc

In [ ]:
!sudo fuser /dev/nvidia*